<center>
<img src="https://user-images.githubusercontent.com/89130737/233078351-bb8134ec-0d1f-495f-a8c0-751e270a7ec0.png" width=250 height=100/>
</center>
<h2 align="center">Auto generate release notes from GitHub</h2>

##### This notebook generates a release notes by the given repository name, *from*/*to* tags and allows you to preview and store the generated release notes. Note that in order to generate release notes, you **must** have a personal access token (a.k.a PAT), and if you don't have yet you can see how to do it [here](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token#creating-a-personal-access-token-classic).

### Set environment

##### Insert your GitHub Token

In [ ]:
# Enter GitHub token ...
import getpass
import os

if (token := os.environ.get("GITHUB_TOKEN")) is None:
    token = getpass.getpass(prompt="Enter your GitHub token: ")
    os.environ["GITHUB_TOKEN"] = token

##### Install requirements

In [ ]:
# Clone repo and install requirements ...
from tempfile import mkdtemp

if "rnotes/docs" not in os.getcwd():
    tmp_dir = mkdtemp()
    %cd {tmp_dir}
    !git clone https://{token}@github.com/dyeheske/rnotes.git
    %cd rnotes
    %pip install -q -r ./docs/requirements.txt
else:
    os.chdir("../")

##### Set imports

In [ ]:
# Imports ...
import sys
import os
import markdown
import webbrowser
from pathlib import Path
from tempfile import mkdtemp
from IPython.display import Markdown, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from rich.console import Console
from rich.text import Text
import pkgutil


sys.path.insert(0, ".")
from rnotes.query import get_github_repository
from rnotes.rnotes import generate_release_notes
from rnotes.utils import init_log, get_file_name

log_path = Path("rnotes.log")
init_log(log_path, level="INFO")

##### Define functions

In [ ]:
# Function definitions ...
console = Console(force_jupyter=True)

try:
    assert pkgutil.find_loader('google.colab') is not None
    def download(path: Path) -> None:
        from google.colab import files
        files.download(path.resolve())
except:
    def download(path: Path) -> None:
        if "DISPLAY" not in os.environ:
            console.log("⚠️ Unable to open the browser in this platform (maybe you are using remote ssh ?)")
            return
        button = widgets.Button(description="Download")
        def on_button_clicked(value):
            root = tk.Tk()
            root.withdraw()
            output_dir = Path(filedialog.askdirectory())
            out_path = output_dir / path.name
            out_path.write_text(path.read_text())
            print(f"Downloaded to: {out_path.resolve()}")
        button.on_click(on_button_clicked)
        display(button)


def run_button(repository: str, from_tag: str, to_tag: str, output_dir: Path, file_name: str) -> None:
    button = widgets.Button(description="Run")
    def on_button_clicked(value):
        with console.status("[bold green]Generating release notes ...") as status:
            try:
                generate_release_notes(
                    repository_name=repository,
                    from_tag=from_tag,
                    to_tag=to_tag,
                    output_dir=output_dir,
                    file_name=file_name,
                )
                console.print(Text("\N{WHITE HEAVY CHECK MARK} Successfully written release notes."))
            except:
                console.print(Text("\N{CROSS MARK} Failed to generate release notes."))
                download(log_path)
    button.on_click(on_button_clicked)
    display(button)


def open_in_browser_button(path: Path) -> None:
    if "DISPLAY" not in os.environ:
        console.log("⚠️ Unable to open the browser in this platform (maybe you are using remote ssh ?)")
        return
    button = widgets.Button(description="Open in browser")
    def on_button_clicked(value):
        default_browser = webbrowser.get().name
        available_browser = webbrowser.get(default_browser)
        available_browser.open_new_tab(path)
    button.on_click(on_button_clicked)
    display(button)


def display_file(path: Path) -> None:
    button = widgets.Button(description="Display")
    def on_button_clicked(value):
        if button.description == "Display":
            if path.suffix == ".md":
                display(Markdown((path).read_text()))
            else:
                display(Text(path.read_text()))
            button.description = "Clear"
        else:
            button.description = "Display"
            clear_output(wait=False)
            display(button)
    button.on_click(on_button_clicked)
    display(button)


def to_html(path: Path, suffix: str = "") -> Path:
    html_file_path = None
    content = path.read_text()
    if path.suffix == ".md":
        html_content = markdown.markdown(content)
    else:
        str_path = str(path.resolve())
        html_content = '<html>\n<head>\n<title>' + str_path + '</title>\n</head>\n<body>\n<pre>\n'
        html_content += content.replace('<', '&lt;').replace('>', '&gt;')
        html_content += '\n</pre>\n</body>\n</html>'
    html_file_path = Path(os.path.splitext(str_path)[0] + f"{suffix}.html")
    html_file_path.write_text(html_content)
    return html_file_path


### Set arguments to rnotes

In [ ]:
# GitHub repository
repository_name = widgets.Combobox(description='Repository:', placeholder='owner/repository', value="dyeheske/dummy_tool")
display(repository_name)

In [ ]:
# Choose tags:
repository = get_github_repository(repository_name=repository_name.value)
tag_names = sorted(repository.tags)
from_tag = widgets.Dropdown(options=tag_names, description='From tag:')
to_tag = widgets.Dropdown(options=tag_names, description='To tag:', value="v0.2" if repository_name.value == "dyeheske/dummy_tool" else None)
display(from_tag)
display(to_tag)

### Run:

In [ ]:
# Run ...
output_dir = Path(mkdtemp())
file_name = get_file_name(tool_name=repository_name.value.split("/")[1], version_name=to_tag.value, sfx=".md")
run_button(repository=repository_name.value, from_tag=from_tag.value, to_tag=to_tag.value, output_dir=output_dir, file_name=file_name)
result_path = output_dir / file_name

### Results:

#### Display the release notes:

In [ ]:
display_file(result_path)

#### Download files:

Download release notes file:

In [ ]:
download(result_path)

Download release notes file as HTML:

In [ ]:
html_path = to_html(result_path)
download(html_path)

Download the log file:

In [ ]:
download(log_path)

Download the log file as HTML:

In [ ]:
html_path = to_html(log_path, suffix="_log")
download(html_path)